In [2]:
#!/usr/bin/python

import pandas as pd
import sys

def key_exclude(fasta_file, fasta_uc_file):
    fasta_all = {}
    with open(fasta_file, "r") as f:
        for line in f.readlines():
            if line[0] == ">":
                key = line[1:-1]
            else:
                fasta_all[key] = line[:-1]
    
    temp = ["type", "cluster", "size", "pctId", "strand", "qlo", "tlo", "alignment", "query", "target"]
    fasta_uc = pd.read_table(fasta_uc_file, sep="\s", names=temp, engine='python')
    fasta_uc = fasta_uc.loc[fasta_uc.type != "C",:]
    
    temp = fasta_uc.type == "S"
    temp = fasta_uc.loc[temp,:].index
    temp = {temp[i-1] : temp[i] - temp[i-1] for i in range(1, len(temp))}
    temp = dict(sorted(temp.items(), key=lambda x: x[1], reverse=True))
    candidate = {item[0]:item[1] for item in temp.items() if item[1] > 26}
    result = []
    
    for key in candidate.keys():
        cluster = fasta_uc.iloc[key:key+candidate[key],:]
        cluster = cluster["query"]
        temp = set([item.split("_")[0] for item in cluster[1:]])
        if len(temp) >= 26:
            for item in cluster[1:]:
                result.append(">"+item+"\n"+"".join(fasta_all[item])+"\n")
    return result
    
if __name__ == "__main__":
    #fasta_file = str( sys.argv[1] )
    #fasta_uc_file = str( sys.argv[2] )
    
    fasta_file = "01_ALL.fasta"
    fasta_uc_file = "01_ALL.fasta.uc"
    
    result = key_exclude(fasta_file, fasta_uc_file)
    with open("exclude.fasta", "w") as f:
        for item in result:
            f.write(item)